## We will be using the final ensemble of these networks for inference. Users can supply their own images as a URL, and the script will produce image examples of the top 3 predictions by the network.

### Install the required packages

In [24]:
# Install required packages
from distutils.dir_util import copy_tree
copy_tree("D:/Capstone Project/results/lib", "D:/Capstone Project/Working/")
!pip install efficientnet_pytorch

import os
import csv
import albumentations as A
from glob import glob
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import sys
sys.path.append("D:/Capstone Project/results/lib")

import torch
from torch.utils.data import DataLoader

In [25]:
from data import CLASSES, format_prosthesis_name
from metrics import print_metrics
from plots import plot_confusion_matrix
from datasets import OrthonetClassificationDataset
from models import get_unet, CLASSIFIER_MODEL_GENERATORS
from training import load_classifier_transforms
from testing import eval_seg, eval_unetseg, eval_ensemble, write_predictions_to_csv

In [29]:
# Paths
CSV_TEST = "D:/Capstone Project/archive/test.csv"
MODEL_DIR = "D:/Capstone Project/Orthonet_Models"
DATA_PATH = "D:/Capstone Project/archive/orthonet data/orthonet data new"

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BS_TEST = 8
N_WORKERS = 2

In [30]:
test_transforms = load_classifier_transforms()
ds_test = OrthonetClassificationDataset('test', CSV_TEST, DATA_PATH, test_transforms)
dl_test = DataLoader(ds_test, BS_TEST, shuffle=False, num_workers=N_WORKERS, pin_memory=True)

ds_test.stats()

class_names = [format_prosthesis_name(c) for c in CLASSES]

Found 180 test samples from D:/Capstone Project/archive/test.csv

156 unique patients
Class                                             Number of samples
Knee_SmithAndNephew_Legion2                       15
Hip_JRIOrtho_FurlongEvolution_NilCol              15
Hip_DepuySynthes_Corail_NilCol                    15
Hip_Stryker_Exeter                                15
Knee_ZimmerBiomet_Oxford                          15
Knee_Depuy_Synthes_Sigma                          15
Hip_SmithAndNephew_Polarstem_NilCol               15
Knee_SmithAndNephew_GenesisII                     15
Hip_DepuySynthes_Corail_Collar                    15
Hip_JRIOrtho_FurlongEvolution_Collar              15
Hip_Stryker_AccoladeII                            15
Hip_SmithAndNephew_Anthology                      15




## Simple Classifier
It uses simple CNN with a single (grayscale) color channel to classify X-rays.7 different architectures of neural network are assessed.

In [31]:
results_by_model_2d = {}
best_2d_model = (0, None, None)  # Will store accuracy, model_type and the path to the weights

for model_type, model_generator in CLASSIFIER_MODEL_GENERATORS.items():
    
    model_paths = glob(os.path.join(MODEL_DIR, f"{model_type}*.pt"))
    if len(model_paths) == 0:
        continue
    
    mdl = model_generator(n_in=1, n_out=len(CLASSES)).to(DEVICE)
    assert len(model_paths) == 1, f"Expected to find 1 {model_type} models, but found {len(model_paths)}"
    mdl.load_state_dict(torch.load(model_paths[0], map_location ='cpu')['state_dict'])
    
    results_2d = eval_seg(mdl, dl_test, DEVICE)
    results_by_model_2d[model_type] = results_2d

    print_metrics(results_2d, model_type.title())
    write_predictions_to_csv(results_2d, f"results_2d_{model_type}.csv")

    acc = accuracy_score(results_2d['y_true'], results_2d['y_pred'])
    if acc > best_2d_model[0]:
        best_2d_model = acc, model_type, model_paths[0]

best_acc_2d, best_model_type_2d, _best_model_path_2d = best_2d_model
cm = confusion_matrix(results_by_model_2d[best_model_type_2d]['y_true'], results_by_model_2d[best_model_type_2d]['y_pred'])
plot_confusion_matrix(cm, class_names, f"Confusion matrix for best simple 2D classifier ({best_model_type_2d.title()})\n"
                                       f"Accuracy {best_acc_2d*100:.1f}%")

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\gnand\anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\gnand\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\gnand\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\Capstone Project/results/lib\datasets.py", line 99, in __getitem__
    img = self.transforms(image=img)['image'] / 255.
TypeError: 'tuple' object is not callable
